### Loading needed .NET nuget packages

First step is to load the needed .NET nuget packages.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.JSON, 8.0.0"

### Import all using statement

Here we will import all needed using statements


In [25]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.Extensions.Configuration;
using System.IO;


### Load appsettings.json

Now we load the appsettings.json, this file is needed to connect to Azure OpenAI, this is needed to get the API Key and Endpoint of Azure OpenAI.  In real production scenario you should use Managed Identity to connect to Azure OpenAI.

In [23]:
string appsettingsFilePath = $"{System.IO.Directory.GetCurrentDirectory()}\\appsettings.json";

var configuration = new ConfigurationBuilder()
                        .AddJsonFile(appsettingsFilePath, optional: false, reloadOnChange: false)                                                     
                        .Build();

7ecf5e600e0745d3b4ebda0221990e66


We are now building the Kernel

In [24]:
var builder = Kernel.CreateBuilder();

// We load the OpenAIChatCompletion but other builder are avaiable
builder.AddAzureOpenAIChatCompletion(deploymentName: configuration["OpenAIConfiguration:DeploymentName"], 
                                     endpoint: configuration["OpenAIConfiguration:OpenAiEndpoint"], 
                                     apiKey: configuration["OpenAIConfiguration:OpenAiKey"]);

// Kernel ready up and running
var kernel = builder.Build();

Load the plugins we will use

In [50]:
// Just needed in case we run the cell multiples time to not have plugin already loaded errors
kernel.Plugins.Clear();

// Load fun fact plugin
var funFactPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(),"Plugins\\FunFact");

var funFactPluginFunctions = kernel.ImportPluginFromPromptDirectory(funFactPluginDirectoryPath);

// Construct arguments
Console.WriteLine("Fun fact about a famous dish from Quebec that is delicious");
Console.WriteLine("");

var arguments = new KernelArguments() { ["input"] = "Fun fact about a famous dish from Quebec that is delicious" };

var result = await kernel.InvokeAsync(funFactPluginFunctions["GetQuebecFact"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);

Console.WriteLine("");
Console.WriteLine("Fun fact about a famous dish from Ontario that is delicious");
Console.WriteLine("");

arguments = new KernelArguments() { ["input"] = "Fun fact about a famous dish from Ontario that is delicious" };

result = await kernel.InvokeAsync(funFactPluginFunctions["GetOntarioFact"], arguments);

Console.WriteLine(result);

Console.WriteLine("");
Console.WriteLine("Fun fact about an animal in quebec and an hunter");
Console.WriteLine("");

// Ask another FunFact
arguments = new KernelArguments() { ["input"] = "Fun fact about an animal in quebec and an hunter" };

result = await kernel.InvokeAsync(funFactPluginFunctions["GetQuebecFact"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);


Error: System.ArgumentException: An item with the same key has already been added. Key: FunFact
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Microsoft.SemanticKernel.KernelPluginCollection.Add(KernelPlugin plugin)
   at Microsoft.SemanticKernel.KernelExtensions.ImportPluginFromPromptDirectory(Kernel kernel, String pluginDirectory, String pluginName, IPromptTemplateFactory promptTemplateFactory)
   at Submission#49.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)